Assignment 3 - Diachronic analysis of movie dialogues
===

*Due: January 1 2023*

In this assignment you will analyze how the expression of Hope in movie dialogues changed with time.

## Imports

In [36]:
from nltk.corpus import wordnet as wn
import pandas as pd
import ipywidgets as widgets
from itertools import compress


layout = widgets.Layout(width="auto")
lang = "eng"
data_path = "data/dialogs_preprocessed2/"

## Functions

In [7]:
def generate_word_list(seed_word, language):

    ## we create an empty list to store the final word list
    list_of_lemmas = []
    list_of_meanings = []

    ## a function to add a word to a list
    add_to_list = lambda list1, item1: list1.append(item1)

    ## a function to return the hyponyms of a synset
    hypos = lambda s: s.hyponyms()

    ## wn.synset obtains the list of synonyms and meanings for that word, in different syntactic categories
    meanings = wn.synsets(seed_word, pos=wn.NOUN + wn.VERB + wn.ADJ)

    ## loop over set of meanings in synset
    for meaning in meanings:

        ## add synset, definition, and a list of all associated lemmas into the list_of_meanings
        list_of_meanings += [
            [
                meaning,
                meaning.definition(),
                [lemma.name() for lemma in meaning.lemmas(language)],
            ]
        ]

        ## append all synonyms (lemmas()) of that meaning to the list_of_lemmas
        [
            add_to_list(list_of_lemmas, lemma.name())
            for lemma in meaning.lemmas(language)
        ]

        ## loop over the list of all possible hyponyms
        for hyponym in meaning.closure(hypos):

            ## add synsets, definition, and a list of all associated lemmas into the list_of_meanings
            list_of_meanings += [
                [
                    hyponym,
                    hyponym.definition(),
                    [lemma.name() for lemma in hyponym.lemmas(language)],
                ]
            ]

            ## append all synonyms (lemmas()) of that hyponym to the list_of_lemmas
            [
                add_to_list(list_of_lemmas, lemma.name())
                for lemma in hyponym.lemmas(language)
            ]

    ##eliminate list duplications by applying the set transformation
    set_of_lemmas = [*set(list_of_lemmas)]

    ## sort alphabetically
    set_of_lemmas.sort()

    ##length
    length = len(set_of_lemmas)

    return (set_of_lemmas, length, list_of_meanings)

## 0. Expression of interest: Hope

In [8]:
seed_word = "hope"

## 1. Control measure: Irritation

Find control measure to contrast with "Hope"

In [40]:
ctrl_seed_word = "irritation"

## 2. Initial bags of seeds

Find psychometric tools or equivalent to generate initial bags of seeds, for both Hope and the control condition

In [41]:
seed_list = generate_word_list(seed_word, lang)
print(f"Length of lemma list is {seed_list[1]}")

Length of lemma list is 16


In [42]:
ctrl_seed_list = generate_word_list(control_seed_word, lang)
print(f"Length of lemma list is {ctrl_seed_list[1]}")

Length of lemma list is 29


## 3. Expand bag of seeds and get synonyms and hyponyms

Use WordNet tools to expand your bag of seeds and get synonyms and hyponyms, and start excluding words with unrelated meanings using the filters available.

### Hope

In [37]:
selection_widget = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in seed_list[2]
    ]
)

selection_widget

In [50]:
def prune_list(w, seed_list):
    filtered_meanings = list(
        compress(seed_list[2], [widget.value for widget in w.children])
    )

    ## now we extract just the lemmas
    filtered_list = [word for lemmas in filtered_meanings for word in lemmas[2]]

    ## eliminate duplications and sort alphabetically

    filtered_list = sorted([*set(filtered_list)])

    return filtered_list


filtered_list = prune_list(selection_widget, seed_list)

print(filtered_list)

['desire', 'encouragement', 'go_for', 'great_white_hope', 'hope', 'hopefulness', 'optimism', 'promise', 'rainbow', 'sanguineness', 'sanguinity', 'trust', 'white_hope']


### Irritation

In [43]:
ctrl_selection_widget = widgets.VBox(
    [
        widgets.Checkbox(
            value=True,
            description=str(item),
            disabled=False,
            indent=False,
            layout=layout,
        )
        for item in ctrl_seed_list[2]
    ]
)

ctrl_selection_widget

In [51]:
ctrl_filtered_list = prune_list(ctrl_selection_widget, ctrl_seed_list)

print(ctrl_filtered_list)

['annoyance', 'annoying', 'botheration', 'bummer', 'discomfort', 'exasperation', 'huff', 'impatience', 'irritation', 'last_straw', 'miff', 'pinprick', 'red_flag', 'restlessness', 'seeing_red', 'snit', 'soreness', 'taunt', 'taunting', 'twit', 'vexation']


### Extend bag of words with new synonyms/hyponyms

In [52]:
#
#
#

## 4. Train semantic vector space model

Train a semantic vector space model using word2vec

In [1]:
#
#
#

## 5. Semantic clouds

Use your vector space model to find out the semantic clouds of each word in your bag of seeds, and select only the words with semantically meaningful clouds

In [1]:
#
#
#

## 6. Final bag of rods and word frequencies

Take the final Bag of Words, prune it further if necessary, and use the second script to calculate word frequencies (don't forget to upload the text files)

In [1]:
#
#
#

## 7. Relevant ratio, visualization of time series

Calculate your psychological relevant ratio, build a dataframe and plot a time series using Seaborn

In [1]:
#
#
#